#Classificação de músicas do Spotify 🎵.

## Sobre a base de dados:🎲


Este é um conjunto de dados de faixas do Spotify em uma variedade de 125 gêneros diferentes. Cada faixa possui alguns recursos de áudio associados a ela.

Uso da base: 🛠️
O conjunto de dados pode ser usado para:



1.   Construindo um sistema de recomendação com base em alguma entrada ou preferência do usuário
2.   Finalidades de classificação com base em recursos de áudio e gêneros disponíveis
3.   Qualquer outro aplicativo que você possa imaginar. Sinta-se à vontade para discutir!






## Objetivo do nosso projeto:🎯

Este notebook contempla a criação de um modelo preditivo para classificar músicas lentas e agitadas utilizando a base de dados "Spotify Tracks Dataset" do Kaggle.

O link desta base e mais detalhes se encontra em: https://www.kaggle.com/datasets/maharshipandya/-spotify-tracks-dataset

In [ ]:
import pandas as pd

In [ ]:
pd.set_option('display.max_columns', None)
df = pd.read_csv('/content/dataset.csv')
df.head()

In [ ]:
df.shape

##Criando a variável target🎯

De acordo com a regra de negócio, a coluna valence é uma medida de 0,0 a 1,0 que descreve a positividade musical transmitida por uma faixa. Faixas com alta valência soam mais positivas (por exemplo, feliz, alegre, eufórica), enquanto faixas com baixa valência soam mais negativas (por exemplo, triste, deprimida, irritada). Para criar a nossa coluna alvo do modelo preditivo, vamos utizar a coluna "valence" para ser nosso critério de músicas agitadas ou lentas.

In [ ]:
import matplotlib.pyplot as plt

#Analisando a colune "valence" com o intuito de criar a variável de target
plt.hist(df['valence'], bins=20, color='red', edgecolor='black')
plt.xlabel('Valence')
plt.ylabel('Frequência')
plt.title("Histograma da coluna Valence")
plt.show()

Uma análise superficial do gráfico demonstra que ele está levemente deslocado para a esquerda, o que pode sinalizar que a maioria das músicas tem um ritmo mais lento, mas a discussão ainda irá seguir adiante.

In [ ]:
df['valence'].describe() #características da coluna

Criando uma função para categorizar os diferentes tipos de música entre agitada e lenta a partir do limiar de valence.

In [ ]:
def categorizar_valence(row):
  if row['valence'] > 0.5:
    return 'agitada'
  else:
    return 'lenta'
#Agora iremos criar a nova coluna de 'target' utilizando a função categorizar_Valence
df['target'] = df.apply(categorizar_valence, axis=1)
df.head()

In [ ]:
df.tail()

## Feature Engineering 🛠️

Como próximo passo, vamos armazenar em um novo dataframe apenas as colunas necessárias para nossa classificação de músicas.

In [ ]:
df.columns

In [ ]:
df_musica = df.drop(['Unnamed: 0', 'track_id'], axis = 1)
df_musica.head()

## Tratando os dados categóricos 🅰️

LabelEncoder: Essa classe é utilizada para codificar rótulos de classes em números inteiros. É frequentemente usado quando se trabalha com algoritmos de aprendizado supervisionado que requerem rótulos numéricos.

In [ ]:
df_musica.head() #visualizando as colunas categóricas

In [ ]:
def label_encoder_dataframe(df, columns_to_encode):
  from sklearn.preprocessing import LabelEncoder
  le = LabelEncoder()

  for column in columns_to_encode:
    if column in df.columns:
      df[column] = le.fit_transform(df[column])
    else:
      print('A lista possui colunas que não existem no DataFrame.')
  return df

colunas_a_codificar = ['artists', 'album_name', 'track_name', 'explicit', 'track_genre', 'target']
label_encoder_dataframe(df_musica, colunas_a_codificar)
df_musica.head() #o dataframe foi atualizado para ter apenas dados numéricos agora.

## Analisando as variáveis que vão compor nosso modelo 📊

Fazendo um gráfico de correlação com o seaborn.

In [ ]:
import seaborn as sns

In [ ]:
correlation_matrix = df_musica.corr().round(2)

fig,ax = plt.subplots(figsize=(12,10))
sns.heatmap(data=correlation_matrix, annot=True, cmap='coolwarm', ax=ax);

Poucas variáveis tem uma alta correlação ao analisar o gráfico de calor, mas é sempre bom lembrar que correlação não é causalidade.

## Entendendo o equilíbrio da target 🎯

In [ ]:
#Vamos verificar o equilíbrio das classes na target
round(df_musica['target'].value_counts(normalize=True)*100,2)

In [ ]:
set(df['target'])

In [ ]:
set(df_musica['target']) #apenas conferindo os valores que 0 e 1 representam: 0 = Agitada e 1 = Lenta

Como as classes estão bem equilibradas não vai ser necessário realizar técnicas adicionais para tratar um possível desequilíbrio, como oversampling e outras.

## Separando os dados em treino e teste 🔄

Agora vamos criar um modelo supervisionado.

In [ ]:
df_musica.columns

In [ ]:
#separando os dados em treino e teste (método hold out - divisão, treinamento e avaliação)
from sklearn.model_selection import train_test_split

X = df_musica[['popularity', 'duration_ms', 'danceability', 'energy', 'loudness', 'acousticness', 'instrumentalness', 'liveness', 'track_genre']]
y = df_musica['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)


In [ ]:
print(X_train.shape, X_test.shape)

## Normalizando os dados 📏

Ao aplicar o MinMaxScaler, todos os valores dos dados serão transformados para o intervalo entre 0 e 1, onde o valor mínimo será 0 e o valor máximo será 1. Essa técnica é especialmente útil quando os algoritmos de aprendizado de máquina são sensíveis à escala dos dados.

Por que aplicamos a normalização dos dados com as bases já separadas em treino e teste? 🤔 Se aplicarmos normalizações antes de dividir em conjuntos de treino e teste, podemos acabar introduzindo informações do conjunto de teste no conjunto de treino. Isso pode levar a uma avaliação otimista do desempenho do modelo, uma vez que o modelo terá visto parte dos dados de teste durante o treinamento. Este tipo de problema também é chamado de Data Leak (vazamento de dados).

In [ ]:
from numpy import ScalarType
from sklearn.preprocessing import MinMaxScaler

#Criar uma instância do MinMaxScaler
scaler = MinMaxScaler()

scaler.fit(X_train) #fazemos o fit apenas na base de treinamento para evitar data leak

X_train_escalonado = scaler.transform(X_train)
X_test_escalonado = scaler.transform(X_test)

## Criando uma função para executar modelos de machine learning 🚀

In [ ]:
def roda_modelo(modelo):

    from sklearn.metrics import roc_curve, roc_auc_score, classification_report

    # Treinando modelo com os dados de treino
    modelo.fit(X_train_escalonado, y_train)

    # Calculando a probabilidade e calculando o AUC
    prob_predic = modelo.predict_proba(X_test_escalonado) # obter as probabilidades associadas às classes previstas para cada instância de dados
    auc = roc_auc_score(y_test, prob_predic[:,1])
    print(f"AUC {auc}")

    # Fazendo a predicao dos dados de teste e calculando o classification report
    predicao = modelo.predict(X_test_escalonado)
    print("\nClassification Report")
    print(classification_report(y_test, predicao))

    print("\nRoc Curve\n")
    # Fazer previsões de probabilidades
    y_pred_probs = modelo.predict_proba(X_test_escalonado)[:, 1]

    # Calcular a curva ROC
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_probs)

    # Calcular a AUC (área sob a curva ROC)
    auc = roc_auc_score(y_test, y_pred_probs)

    # Plotar a curva ROC
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='blue', label=f'ROC curve (AUC = {auc:.2f})') # linewidth
    plt.plot([0, 1], [0, 1], color='gray',linestyle='--')
    plt.xlabel('Taxa de Falso Positivo')
    plt.ylabel('Taxa de Verdadeiro Positivo')
    plt.title('Curva ROC')
    plt.legend(loc='lower right')
    plt.show()

    # Converter probabilidades em classes preditas (0 ou 1)
    y_pred = (y_pred_probs > 0.5).astype(int)


## Regressão logística

O modelo se baseia em uma função logística, que transforma as variáveis independentes em uma probabilidade entre 0 e 1. Para novas entradas de dados, o modelo calcula a probabilidade do evento binário ocorrer.

In [ ]:

from sklearn.linear_model import LogisticRegression
modelo_logistico = LogisticRegression()
roda_modelo(modelo_logistico)

Analisando o f1-score e as outras métricas de avaliação, é possível concluir que esse método de regressão não ficou bem adequado com a construção de um bom modelo com os dados selecionados.

## KNN (K-Nearest Neighbors)

Para um novo ponto de dados, o KNN identifica os K pontos mais próximos (vizinhos) no conjunto de treinamento. A classe do novo ponto é a classe mais frequente entre os K vizinhos.

In [ ]:

from sklearn.neighbors import KNeighborsClassifier
modelo_knn = KNeighborsClassifier(n_neighbors=3)
roda_modelo(modelo_knn)


Da mesma forma que o modelo de regressão, esse não ficou tão satisfatório.

## Random Florest

O Random Forest Classifier é um algoritmo de ensemble learning, que combina vários modelos para melhorar a performance. O modelo cria uma floresta de árvores de decisão, onde cada árvore é treinada em um subconjunto aleatório dos dados (bootstrap). A classe final do novo ponto de dados é a classe mais votada pelas árvores da floresta.

In [ ]:

from sklearn.ensemble import RandomForestClassifier
modelo_random_forest = RandomForestClassifier(max_depth=7, n_estimators= 100)
roda_modelo(modelo_random_forest)

Está melhorando aos poucos, mas mesmo assim ainda não tem um rendimento ideal. Vamos utilizar Grid Search para alterar parâmetros.

## Testando novos parâmetros com Grid Search

In [ ]:
from sklearn.model_selection import GridSearchCV

# Defina os parâmetros a serem testados
param_grid = {
    "n_estimators": [100, 200, 300],
    "max_depth": [5, 10, 15]
}

# GridSearchCV
grid_search = GridSearchCV(RandomForestClassifier(), param_grid, cv=5, scoring='f1', n_jobs=1)

# Ajuste o modelo ao conjunto de dados
grid_search.fit(X_train_escalonado, y_train)

rf_params = grid_search.best_params_
print("Melhores hiperparâmetros:", rf_params)

Melhores Hiperparâmetros:{'max_depth':15, 'n_estimators':300}

Testando agora com esses parâmetros

In [ ]:
modelo_random_forest = RandomForestClassifier(max_depth=15, n_estimators= 300)
roda_modelo(modelo_random_forest)

In [ ]:
# Tentando novos parâmetros ajustados manualmente
modelo_random_forest_manual = RandomForestClassifier(n_estimators=500, max_depth=20, min_samples_split=5, random_state=42)
roda_modelo(modelo_random_forest_manual)

Neste exemplo, aumentamos o `n_estimators` (número de árvores) para 500, a `max_depth` (profundidade máxima das árvores) para 20 e adicionamos `min_samples_split=5`. Esses são apenas alguns ajustes que poderiam ser testados manualmente. O `random_state` foi adicionado para garantir a reprodutibilidade dos resultados.

Ainda não está próximo de uma acurácia de 90%, mas melhorou bastante. Provavelmente é melhor trabalhar com mais hiperparâmetros ainda para melhorar esse modelo.

## Testando o Modelo ✅

In [ ]:
import numpy as np

novos_dados = pd.read_excel("novos_dados.xlsx")
base_original = pd.read_excel("novos_dados.xlsx")

#Criando a pipeline
coluna = ['track_genre']
label_encoder_dataframe(novos_dados, coluna)
novos_dados = scaler.transform(novos_dados)

# Realize a previsão usando o modelo Random Forest treinado
previsoes = modelo_random_forest_manual.predict(novos_dados)

# Obtendo o predict
def mapear_valor(valores):
    resultados = []
    for valor in valores:
        if valor == 0:
            resultados.append('Música agitada')
        elif valor == 1:
            resultados.append('Música lenta')
        else:
            resultados.append('Desconhecido')
    return np.array(resultados)

base_original['target'] = mapear_valor(previsoes)
base_original.head()

Com o modelo é possível ver o resultado a partir do track_genre em conjunto com o target, sendo possível visualizar se é uma música agitada ou lenta.